In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

Целевая переменная - price_doc

Задача: добиться наилучшего значения метрики MSE.

In [2]:
df = pd.read_csv('housing.csv')
df

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,sub_area,area_m,...,metro_min_walk,metro_km_walk,mkad_km,kremlin_km,green_part_1000,prom_part_1000,office_count_1000,trc_count_1000,leisure_count_1000,price_doc
0,38,18.0,1.0,14.0,panel,1971.0,1.0,9.0,Dmitrovskoe,7.126815e+06,...,27.084184,2.257015,2.442781,14.856442,17.16,10.33,2,4,0,5150000
1,41,17.0,14.0,16.0,panel,1978.0,1.0,10.0,Savelovskoe,2.641243e+06,...,13.719174,1.143264,11.528984,5.323699,0.90,39.48,17,3,0,5980000
2,41,20.0,11.0,14.0,panel,1989.0,1.0,8.0,Krjukovo,1.084231e+07,...,276.453594,23.037800,20.828299,38.988909,7.54,6.64,0,2,0,4600000
3,38,19.0,6.0,17.0,panel,1986.0,1.0,8.0,Brateevo,7.587523e+06,...,14.085660,1.173805,2.885041,14.921056,24.97,0.00,0,5,0,5650000
4,58,37.0,3.0,9.0,panel,1968.0,3.0,6.0,Novogireevo,4.395333e+06,...,5.455795,0.454650,1.920884,11.812614,3.46,5.41,0,4,4,9300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8889,51,32.0,3.0,12.0,breezeblock,1974.0,2.0,9.0,Pechatniki,1.844581e+07,...,17.320003,1.443334,6.756490,9.929204,13.61,4.32,1,0,0,8300000
8890,76,45.0,6.0,16.0,panel,2001.0,3.0,10.0,Mar'ino,1.175477e+07,...,23.956840,1.996403,3.393475,13.991498,19.97,0.00,0,5,0,12800000
8891,78,43.0,15.0,17.0,panel,2002.0,3.0,13.0,Severnoe Butovo,8.889467e+06,...,3.497793,0.291483,1.056678,19.965501,20.83,0.00,0,7,0,15500000
8892,54,28.0,7.0,16.0,breezeblock,1975.0,2.0,12.0,Ivanovskoe,1.020722e+07,...,30.095498,2.507958,0.795735,13.120257,26.06,0.16,0,2,0,8350000


In [3]:
df.info() # есть ли пропуски

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8894 entries, 0 to 8893
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full_sq             8894 non-null   int64  
 1   life_sq             8894 non-null   float64
 2   floor               8894 non-null   float64
 3   max_floor           8894 non-null   float64
 4   material            8894 non-null   object 
 5   build_year          8859 non-null   float64
 6   num_room            8894 non-null   float64
 7   kitch_sq            8894 non-null   float64
 8   sub_area            8894 non-null   object 
 9   area_m              8894 non-null   float64
 10  green_zone_part     8894 non-null   float64
 11  indust_part         8894 non-null   float64
 12  preschool           8894 non-null   int64  
 13  school              8894 non-null   int64  
 14  healthcare          8894 non-null   int64  
 15  shopping            8894 non-null   int64  
 16  office

In [4]:
df.isna().sum()  #суммарное кол-во пропусков

full_sq                0
life_sq                0
floor                  0
max_floor              0
material               0
build_year            35
num_room               0
kitch_sq               0
sub_area               0
area_m                 0
green_zone_part        0
indust_part            0
preschool              0
school                 0
healthcare             0
shopping               0
office                 0
radiation              0
detention              0
young                  0
work                   0
elder                  0
0_6_age                0
7_14_age               0
metro_min_avto         0
metro_km_avto          0
metro_min_walk        11
metro_km_walk         11
mkad_km                0
kremlin_km             0
green_part_1000        0
prom_part_1000         0
office_count_1000      0
trc_count_1000         0
leisure_count_1000     0
price_doc              0
dtype: int64

In [5]:
df = df.drop_duplicates().reset_index(drop=True) # удалим дубликаты
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8852 entries, 0 to 8851
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full_sq             8852 non-null   int64  
 1   life_sq             8852 non-null   float64
 2   floor               8852 non-null   float64
 3   max_floor           8852 non-null   float64
 4   material            8852 non-null   object 
 5   build_year          8817 non-null   float64
 6   num_room            8852 non-null   float64
 7   kitch_sq            8852 non-null   float64
 8   sub_area            8852 non-null   object 
 9   area_m              8852 non-null   float64
 10  green_zone_part     8852 non-null   float64
 11  indust_part         8852 non-null   float64
 12  preschool           8852 non-null   int64  
 13  school              8852 non-null   int64  
 14  healthcare          8852 non-null   int64  
 15  shopping            8852 non-null   int64  
 16  office

In [6]:
df = df.dropna() # удалим пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8806 entries, 0 to 8851
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   full_sq             8806 non-null   int64  
 1   life_sq             8806 non-null   float64
 2   floor               8806 non-null   float64
 3   max_floor           8806 non-null   float64
 4   material            8806 non-null   object 
 5   build_year          8806 non-null   float64
 6   num_room            8806 non-null   float64
 7   kitch_sq            8806 non-null   float64
 8   sub_area            8806 non-null   object 
 9   area_m              8806 non-null   float64
 10  green_zone_part     8806 non-null   float64
 11  indust_part         8806 non-null   float64
 12  preschool           8806 non-null   int64  
 13  school              8806 non-null   int64  
 14  healthcare          8806 non-null   int64  
 15  shopping            8806 non-null   int64  
 16  office     

In [7]:
# закодируем категориальные переменные
label_enc={}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_enc[col] = le

In [8]:
# опытным путем оказалось, что наименьшая MSE оказалась в случае линейной регресии при разделении данных на 33/67
# ниже представлен результат

X = df.drop(['price_doc'], axis=1)
y = df[['price_doc']]
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=42)

sc=StandardScaler() # стандартизируем
x_train=sc.fit_transform(train_X)
x_test=sc.fit_transform(test_X)

reg = LinearRegression().fit(x_train, train_y)

pred = reg.predict(x_test)

mse = mean_squared_error(test_y, pred)
r2 = r2_score(test_y, pred)
print(f'MSE: {round(mse,3)}\nR^2: {round(r2,3)}')

MSE: 10249449139147.305
R^2: 0.518
